In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
import numpy as np

print("--- 学習と予測のセットアップ開始 ---")

# --- 1. Load training data ---
print("--- 学習データを読み込み中 ---")
# パスはKaggleの推奨通り絶対パスで指定します
seqs   = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')
labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv') # x_1, y_1, z_1 のみを持つデータ
print("学習データの読み込み完了。")

# --- 2. Merge sequence into labels ---
print("--- シーケンスをラベルにマージ中 ---")

labels['target_id'] = labels['ID'].str.rpartition('_')[0]

# train_labels.csvはID, resname, resid, x_1, y_1, z_1列を持つことを確認済みです。
# そのため、そのままseqsとマージします。
train = labels.merge(
    seqs[['target_id','sequence']],
    on='target_id', how='left'
)
train['sequence_length'] = train['sequence'].str.len()

print("シーケンスのマージ完了。")

# --- 3. Features and targets ---
print("--- 特徴量とターゲットを準備中 ---")

# ターゲットは x_1, y_1, z_1 のみ (学習データから得られる唯一の座標)
target_coord_cols = ['x_1', 'y_1', 'z_1']

# 特徴量に使用する列を定義
feature_cols = ['resid', 'sequence_length']
one_hot_cols = ['resname'] # One-hotエンコーディング対象

# X を作成
X = pd.get_dummies(train[feature_cols + one_hot_cols], columns=one_hot_cols, prefix='res')

# ターゲットは x_1, y_1, z_1
y = train[target_coord_cols]

# 欠損値を含む行を削除 (Xとyで共通の行を保持)
# y_clean の NaN がある行を削除します。
mask = ~y.isna().any(axis=1)
X_clean = X[mask].reset_index(drop=True)
y_clean = y[mask].reset_index(drop=True)

print(f"クリーンアップ後のデータサイズ: {len(X_clean)} 行")
print("特徴量とターゲットの準備完了。")

# --- 4. Prepare test features ---
print("\n--- テスト特徴量を準備中 ---")
test_seqs = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')
rows = []
for _, row in test_seqs.iterrows():
    tid, seq = row['target_id'], row['sequence']
    L = len(seq)
    for i, nt in enumerate(seq, start=1):
        rows.append({
            'ID': f"{tid}_{i}",
            'resid': i,
            'sequence_length': L,
            'resname': nt # test_sequences から resname を取得
        })
test_df_base = pd.DataFrame(rows) # このDataFrameに最終予測を追加していく

# One-hot encode and align with X_clean columns
X_test_processed = pd.get_dummies(test_df_base[feature_cols + one_hot_cols], columns=one_hot_cols, prefix='res')

# X_clean のカラムリストに基づいてX_test_processedの列を調整
X_clean_feature_cols = X_clean.columns.tolist()

# X_clean にあるが X_test_processed にない特徴量列を追加し、0で埋める
for col in X_clean_feature_cols:
    if col not in X_test_processed.columns:
        X_test_processed[col] = 0

# X_test_processed にあるが X_clean にない特徴量列を削除
for col in X_test_processed.columns:
    if col not in X_clean_feature_cols:
        X_test_processed = X_test_processed.drop(columns=[col])

# 最終的なX_test_processedの列順をX_clean_feature_colsに合わせる
X_test_aligned = X_test_processed[X_clean_feature_cols]

print("テスト特徴量の準備完了。")

# --- 5. Train and Predict with K-Fold Cross-Validation (各フォールドの予測を異なる原子座標に割り当て) ---
print("\n--- K分割交差検証でモデルを学習・予測中 ---")
print("--- 各フォールドで予測された x_1,y_1,z_1 が、x_N,y_N,z_N に割り当てられます ---")
n_splits = 5 # 5つのモデルを学習し、5つの座標グループに割り当てます
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# 提出に必要な全ての座標の列名を定義 (x_1からz_5まで)
full_submission_coord_cols = [f'{axis}_{num}' for num in range(1, 6) for axis in ['x', 'y', 'z']]

# 最終的な予測結果を格納するDataFrameを準備
# test_df_base のコピーに、全ての座標列を追加しておきます
final_prediction_df = test_df_base.copy()
for col in full_submission_coord_cols:
    final_prediction_df[col] = np.nan # 初期値はNaNで埋めておきます

# 各フォールドで学習したモデルの予測結果を、対応する原子座標の列に格納
for fold, (train_index, val_index) in enumerate(kf.split(X_clean)):
    # fold は 0 から n_splits-1 までの値を取ります
    # これを原子のインデックス (1からn_splits) に対応させます
    atom_idx_for_this_fold = fold + 1

    print(f"\n--- フォールド {fold + 1}/{n_splits} (予測を x_{atom_idx_for_this_fold},y_{atom_idx_for_this_fold},z_{atom_idx_for_this_fold} に割り当て) ---")
    X_train_fold, y_train_fold = X_clean.iloc[train_index], y_clean.iloc[train_index]

    model_fold = MultiOutputRegressor(GradientBoostingRegressor(random_state=42 + fold)) # 各モデルのrandom_stateを変えて多様性を持たせる
    model_fold.fit(X_train_fold, y_train_fold)
    print(f"フォールド {fold + 1} のモデル学習完了。")

    # このフォールドのモデルでテストデータを予測 (preds_fold は x_1, y_1, z_1 の予測結果)
    preds_fold_xyz1 = model_fold.predict(X_test_aligned)
    print(f"フォールド {fold + 1} のテストデータ予測完了。")

    # このフォールドの予測結果を、対応する原子の座標の列に割り当てます
    # 例: fold 0 の予測 -> x_1, y_1, z_1 に割り当て
    #     fold 1 の予測 -> x_2, y_2, z_2 に割り当て (これは元データにはないが、提出用)
    cols_to_assign = [f'x_{atom_idx_for_this_fold}', f'y_{atom_idx_for_this_fold}', f'z_{atom_idx_for_this_fold}']
    final_prediction_df[cols_to_assign] = preds_fold_xyz1

print("\nK-Fold交差検証による全ての原子座標の割り当て完了。")

# --- 6. Build and save submission ---
print("\n--- 提出ファイルを構築・保存中 ---")
# 提出に必要な列を最終的なDataFrameから選択します
submission = final_prediction_df[['ID', 'resname', 'resid'] + full_submission_coord_cols]

out_path = '/kaggle/working/submission.csv' # submission.csv として保存
submission.to_csv(out_path, index=False)
print(f"提出ファイルを {out_path} に保存しました。")
print("\n提出ファイルのhead (最初の5行):")
print(submission.head())

print("\n--- 全処理完了 ---")

--- 学習と予測のセットアップ開始 ---
--- 学習データを読み込み中 ---
学習データの読み込み完了。
--- シーケンスをラベルにマージ中 ---
シーケンスのマージ完了。
--- 特徴量とターゲットを準備中 ---
クリーンアップ後のデータサイズ: 130950 行
特徴量とターゲットの準備完了。

--- テスト特徴量を準備中 ---
テスト特徴量の準備完了。

--- K分割交差検証でモデルを学習・予測中 ---
--- 各フォールドで予測された x_1,y_1,z_1 が、x_N,y_N,z_N に割り当てられます ---

--- フォールド 1/5 (予測を x_1,y_1,z_1 に割り当て) ---
フォールド 1 のモデル学習完了。
フォールド 1 のテストデータ予測完了。

--- フォールド 2/5 (予測を x_2,y_2,z_2 に割り当て) ---
フォールド 2 のモデル学習完了。
フォールド 2 のテストデータ予測完了。

--- フォールド 3/5 (予測を x_3,y_3,z_3 に割り当て) ---
フォールド 3 のモデル学習完了。
フォールド 3 のテストデータ予測完了。

--- フォールド 4/5 (予測を x_4,y_4,z_4 に割り当て) ---
フォールド 4 のモデル学習完了。
フォールド 4 のテストデータ予測完了。

--- フォールド 5/5 (予測を x_5,y_5,z_5 に割り当て) ---
フォールド 5 のモデル学習完了。
フォールド 5 のテストデータ予測完了。

K-Fold交差検証による全ての原子座標の割り当て完了。

--- 提出ファイルを構築・保存中 ---
提出ファイルを /kaggle/working/submission.csv に保存しました。

提出ファイルのhead (最初の5行):
        ID resname  resid        x_1        y_1        z_1        x_2  \
0  R1107_1       G      1  57.209827  62.162951  57.370064  58.209482   
1  R1107_2       G      2  57.209827  62.16295

In [2]:
submission

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,57.209827,62.162951,57.370064,58.209482,61.302028,56.009821,59.577707,62.919033,56.876842,57.352432,62.125393,56.948246,57.846023,63.522751,56.594064
1,R1107_2,G,2,57.209827,62.162951,57.370064,58.209482,61.302028,56.009821,59.577707,62.919033,56.876842,57.352432,62.125393,56.948246,57.846023,63.522751,56.594064
2,R1107_3,G,3,57.209827,62.162951,57.370064,58.209482,61.302028,56.009821,59.577707,62.919033,56.876842,57.352432,62.125393,56.948246,57.846023,63.522751,56.594064
3,R1107_4,G,4,57.209827,62.162951,57.370064,58.209482,61.302028,56.009821,59.577707,62.919033,56.876842,57.352432,62.125393,56.948246,57.846023,63.522751,56.594064
4,R1107_5,G,5,57.209827,62.162951,57.370064,58.209482,61.302028,56.009821,59.577707,62.919033,56.876842,57.352432,62.125393,56.948246,57.846023,63.522751,56.594064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,111.846112,105.063245,108.402664,111.454132,103.918360,109.139944,111.191244,104.034618,108.919609,110.676881,103.822399,111.561845,111.594148,104.216178,108.339962
2511,R1190_115,U,115,111.846112,105.063245,108.402664,111.454132,103.918360,109.139944,111.191244,104.034618,108.919609,110.676881,103.822399,111.561845,111.594148,104.216178,108.339962
2512,R1190_116,U,116,111.846112,105.063245,108.402664,111.454132,103.918360,109.139944,111.191244,104.034618,108.919609,110.676881,103.822399,111.561845,111.594148,104.216178,108.339962
2513,R1190_117,U,117,111.846112,105.063245,108.402664,111.454132,103.918360,109.139944,111.191244,104.034618,108.919609,110.676881,103.822399,111.561845,111.594148,104.216178,108.339962


In [3]:
import os
import pandas as pd

# submission.csv が保存される予定のパス
submission_path = '/kaggle/working/submission.csv'

print(f"submission.csv が保存される想定パス: {submission_path}")

# ファイルが存在するか確認
if os.path.exists(submission_path):
    print(f"ファイル {submission_path} が存在します。")
    try:
        # ファイルを読み込んで先頭を表示し、中身を確認
        submission_df_check = pd.read_csv(submission_path)
        print("\n--- 保存された submission.csv の内容 (head) ---")
        print(submission_df_check.head())
        print(f"\n--- 保存された submission.csv の行数: {len(submission_df_check)} ---")
    except Exception as e:
        print(f"エラー: {submission_path} の読み込み中に問題が発生しました: {e}")
else:
    print(f"エラー: ファイル {submission_path} が見つかりません。")
    print("ノートブックの実行が最後まで成功したか、またはパスが正しいか確認してください。")

submission.csv が保存される想定パス: /kaggle/working/submission.csv
ファイル /kaggle/working/submission.csv が存在します。

--- 保存された submission.csv の内容 (head) ---
        ID resname  resid        x_1        y_1        z_1        x_2  \
0  R1107_1       G      1  57.209827  62.162951  57.370064  58.209482   
1  R1107_2       G      2  57.209827  62.162951  57.370064  58.209482   
2  R1107_3       G      3  57.209827  62.162951  57.370064  58.209482   
3  R1107_4       G      4  57.209827  62.162951  57.370064  58.209482   
4  R1107_5       G      5  57.209827  62.162951  57.370064  58.209482   

         y_2        z_2        x_3        y_3        z_3        x_4  \
0  61.302028  56.009821  59.577707  62.919033  56.876842  57.352432   
1  61.302028  56.009821  59.577707  62.919033  56.876842  57.352432   
2  61.302028  56.009821  59.577707  62.919033  56.876842  57.352432   
3  61.302028  56.009821  59.577707  62.919033  56.876842  57.352432   
4  61.302028  56.009821  59.577707  62.919033  56.876842  57.35